In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import glob
import math
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

label_encoder = LabelEncoder()

### Preparing


In [2]:
route_path = "data/csv/relationship_routes.csv"
relationship_route = pd.read_csv(route_path)

In [3]:
df_process = pd.read_csv("process-time.csv")
missing_rows = pd.read_csv("missing_rows.csv")
df_routes_data = pd.concat([df_process, missing_rows], ignore_index=True)

df_routes_data["ts1"] = pd.to_datetime(df_routes_data["ts1"])
df_routes_data["ts2"] = pd.to_datetime(df_routes_data["ts2"])

# def get_day_of_week(timestamp):
#     return timestamp.day_name()

df_routes_data[(df_routes_data['route_id'] == '1-38') & (~df_routes_data['mins'].isna())]

FileNotFoundError: [Errno 2] No such file or directory: 'process-time.csv'

In [ ]:
df_routes_data.drop(columns=["lat", "lon"], inplace=True)

In [ ]:
# df_routes_data = df_routes_data[df_routes_data['route_id'] == '2-23']

df_routes_data['route_id_encoded'] = label_encoder.fit_transform(df_routes_data['route_id'])

df_routes_data['weekend'] = np.where((df_routes_data['day_of_week'] == 'Saturday') | (df_routes_data['day_of_week'] == 'Sunday'), True, False)

day_to_num = {
    "Monday": 1,
    "Tuesday": 2,
    "Wednesday": 3,
    "Thursday": 4,
    "Friday": 5,
    "Saturday": 6,
    "Sunday": 7,
}

# def map_time_periods(hour):
#     if 4 <= hour <= 8:
#         return 0
#     elif 8 <= hour <= 10:
#         return 1
#     elif 10 <= hour <= 12:
#         return 0
#     elif 12 <= hour <= 15:
#         return 0
#     elif 15 <= hour <= 20:
#         return 1
#     else:
#         return 0
def map_time_periods(hour):
    if 6 <= hour <= 10:  # Merged the first two conditions as they return the same value
        return 'peak'
    elif 10 < hour <= 12:
        return 'off-peak'
    elif 12 < hour <= 15:
        return 'off-peak'
    elif 15 < hour <= 20:
        return 'peak'
    else:
        return 'off-peak'

df_routes_data["time_periods"] = df_routes_data["hrs"].apply(map_time_periods)

df_routes_data["day_of_week"] = df_routes_data["day_of_week"].map(day_to_num)

df_routes_data.sort_values(by=["seq", "day_of_week", "time_periods"], inplace=True)

df_routes_data = df_routes_data.reset_index(drop=True)
df_routes_data

weight travel time for each neighboring node

In [ ]:
# def linear_weight(t_diff):
#     return 1 / (1 + t_diff)

# # Calculate time difference
# df['time_difference_k'] = (pd.to_datetime(df['ts1'].dt.strftime('%H:%M:%S')) - pd.to_datetime(df['hrs'].astype(str) + ':00')).dt.total_seconds() / 60

# # Calculate weights using the linear weight function
# df['weight_k'] = df['time_difference_k'].apply(linear_weight)

# # Calculate weighted travel time for each neighboring node
# df['weighted_travel_time_k'] = df['weight_k'] * df['mins']

# # # Calculate the numerator and denominator for the final calculation
# numerator = df.groupby(['sid1', 'sid2', 'route_id'])['weighted_travel_time_k'].sum()
# denominator = df.groupby(['sid1', 'sid2', 'route_id'])['weight_k'].sum()

# df_result = numerator / denominator

# df = pd.merge(df, df_result.reset_index(), on=['sid1', 'sid2', 'route_id'], how='left')

# df.rename(columns={0:'weighted_travel_time_route'}, inplace=True)

historical average travel time

In [ ]:

# filtered_df = df[(df['sid1'] == 4151) & (df['sid2'] == 4142) & (df['route_id'] == '2-23')]

# historical_average = filtered_df['mins'].mean()

# print(f"The historical average travel time between stops i and j is {historical_average} minutes.")


In [ ]:
# df2 = df[(df["route_id"] == '2-16') & (df["sid1"] == 3851) & (df["sid2"] == 3776) & (df["hrs"] == 4)]

# for bus in df2['vid'].unique():
#    plt.plot(df2[df2['vid'] == bus]['mins'], marker='', linestyle='-', label=bus)


# plt.xlabel('ts1')
# plt.ylabel('mins')
# plt.title('Line Graph Example')
# plt.legend()

# df2

### Data preprocessing

In [ ]:
# df2 = df.reset_index()["mins"]
# plt.plot(df2)

Tranformation


In [ ]:
# result = adfuller(df["mins"])
# print("ADF Statistic: %f" % result[0])
# print("p-value: %f" % result[1])

In [ ]:
# df["mins_diff"] = df["mins"].diff()
# df = df.dropna()  # Differencing leads to NaN values for the first row, so drop it
# df

In [ ]:
# df["mins_log"] = np.log(df["mins"])
# df["mins_log_diff"] = df["mins_log"].diff()
# df = df.dropna()
# df

In [ ]:
# seasonal_difference = df["mins"].diff(24)

In [ ]:
# seasonal_difference = df["mins_diff"].diff(24)

In [ ]:
# result = adfuller(df["mins_log_diff"].dropna(), autolag="AIC")

# print('ADF Statistic:', result[0])
# print("p-value:", result[1])

Encode


In [ ]:
df = df_routes_data[(~df_routes_data["mins"].isna())]
df_test = df_routes_data[(df_routes_data["mins"].isna())][:10000]

# Feature selection
features = [
    "sid1",
    "sid2",
    # "speed",
    "route_id_encoded",
    "direction",
    "seq",
    "hrs",
    "day_of_week",
    "weekend",
    "time_periods",
    # "weighted_travel_time_route",
]
target = "mins"

#################################################
# Select features and target variable
X = df[features]
y = df[target]

# X['route_id_encoded'] = label_encoder.transform(X['route_id'])

X_encoded = pd.get_dummies(
    X,
    columns=[
        "direction",
        "time_periods",
    ],
    drop_first=True,
)

#################################################

X_new = df_test[features]
y_new = df_test["mins"]

X_new_encoded = pd.get_dummies(
    X_new,
    columns=[
        "direction",
        "time_periods",
    ],
    drop_first=True,
)

###################################################

X_encoded
# X_new_encoded

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.ensemble import RandomForestRegressor

# # Assuming X is your feature matrix and y is the continuous target variable
# model = RandomForestRegressor()

# # Perform K-Fold Cross-validation for Regression
# cv_results = cross_val_score(model, X_encoded, y, cv=8, scoring='neg_mean_squared_error')

# # Print the average mean squared error
# print(f'Average Mean Squared Error: {-cv_results.mean()}')

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, feat_name=None):

    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [f"{feat_name[j]}(t-{i})" for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [f"{feat_name[j]}(t)" for j in range(n_vars)]
        else:
            names += [f"{feat_name[j]}(t+{i})" for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# reframed = series_to_supervised(X_encoded.values, 30, 3, feat_name=X_encoded.columns)
# reframed.drop(['High(t)','High(t+1)','High(t+2)','Low(t)','Low(t+1)','Low(t+2)','Open(t)','Open(t+1)','Open(t+2)','Volume(t)','Volume(t+1)',\
#                'Volume(t+2)'],axis=1,inplace=True)

Train-Test Split


In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

Standradize


In [ ]:
# Standardize the data
# scaler_X = StandardScaler()
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# scaler_y = StandardScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = scaler_y.fit_transform(np.array(y_train).reshape(-1, 1))
y_test_scaled = scaler_y.transform(np.array(y_test).reshape(-1, 1))

y_test_scaled.shape

In [ ]:
# X_new_scaled = scaler_X.transform(X_new_encoded)

# # scaler_y = StandardScaler()
# # y_new_scaled = scaler_y.fit_transform(np.array(y_new).reshape(-1, 1))
# # y_new_scaled = scaler_y.transform(np.array(y_new).reshape(-1, 1))
# X_new_scaled.shape

Reshape data for LSTM input (assuming sequence length of 1)

reshape input to be 3D [samples, timesteps, features]


In [ ]:
X_train_lstm = X_train_scaled.reshape(
    (X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
)
X_test_lstm = X_test_scaled.reshape(
    (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

X_train_lstm.shape

In [ ]:
# X_new_lstm = X_new_scaled.reshape(
#     (X_new_scaled.shape[0], 1, X_new_scaled.shape[1]))

# X_new_lstm.shape

Creating and fitting LSTM model


LSTM basic

In [ ]:
# design network
model = Sequential()
model.add(
    LSTM(
        128,
        return_sequences=True,
        activation="relu",
        input_shape=(X_train_lstm.shape[1], X_test_lstm.shape[2]),
    )
)


model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dense(1, activation='relu'))
model.compile(optimizer="adam", loss="mse")

model.summary()

LSTM with drop and Dense('relu')

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense, Add, Activation, Dropout
# from tensorflow.keras import regularizers

# def create_residual_model(input_shape):
#     inputs = Input(shape=input_shape)
    
#     # Initial LSTM layer
#     x = LSTM(128, return_sequences=False)(inputs)
#     x = Dropout(0.5)(x)  # Apply dropout
#     # First dense block
#     x_shortcut = x
#     x = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
#     x = Dense(1, kernel_regularizer=regularizers.l2(0.01))(x)
    
#     # Add residual connection
#     x = Add()([x, x_shortcut])
#     x = Activation('relu')(x)
    
#     # Additional dense blocks as needed with residual connections
#     for _ in range(3 - 1):  # Define your desired number of residual blocks
#         x_shortcut = x
#         x = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
#         x = Dense(1, kernel_regularizer=regularizers.l2(0.01))(x)
        
#         # Add residual connection
#         x = Add()([x, x_shortcut])
#         x = Activation('relu')(x)


#     # x = BatchNormalization()(x)
#     # Output layer
#     outputs = Dense(1, activation='relu')(x)  # Assuming a regression task
    
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

# input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])  # Replace with your input shape
# model = create_residual_model(input_shape)

# model.compile(optimizer='adam', loss='mse')

# model.summary()

LSTM with drop and Dense('tanh')

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense, Add, Activation, Dropout
# from tensorflow.keras import regularizers

# def create_advanced_model(input_shape):
#     inputs = Input(shape=input_shape)
    
#     # Initial LSTM layer
#     x = LSTM(128, return_sequences=False)(inputs)
#     # Apply dropout

#     # First dense block with residual connection
#     x_shortcut = x
#     x = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
#     x = Dense(1, kernel_regularizer=regularizers.l2(0.01))(x)
    
#     # Add residual connection
#     x = Add()([x, x_shortcut])
#     x = Activation('relu')(x)
    
#     # Additional dense blocks with dropout, L2 regularization, and residual connections
#     for _ in range(number_of_residual_blocks - 1):  # Define the desired number of residual blocks
#         x_shortcut = x
#         x = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
#         x = Dense(1, kernel_regularizer=regularizers.l2(0.01))(x)
        
#         # Add residual connection
#         x = Add()([x, x_shortcut])
#         x = Activation('relu')(x)
        
#     x = BatchNormalization()(x)
#     # Output layer with tanh activation
#     outputs = Dense(1, activation='tanh')(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

# input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])  # Replace with your input shape
# number_of_residual_blocks = 3  # Define the number of residual blocks you desire
# model = create_advanced_model(input_shape)

# model.compile(optimizer='adam', loss='mse')

# model.summary()

Define call back

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cp = ModelCheckpoint('model_LSTM_drop/', save_best_only=True)


In [ ]:
history = model.fit(
    X_train_lstm,
    y_train_scaled,
    epochs=50,
    batch_size=64,
    validation_data=(X_test_lstm, y_test_scaled),
    verbose=1,
    callbacks=[
        # early_stopping, 
               cp
               ],
)

In [ ]:
# พล็อตค่า train&test loss
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="test")
plt.legend()
plt.show()

Prediction and checking performance matrix


In [ ]:
model = load_model('model_LSTM_drop/')

In [ ]:
new_predictions_scaled = model.predict(X_test_lstm)
new_predictions_scaled.shape

In [ ]:
# # transform to original form
# scaler_y = StandardScaler()
# y_train_scaled = scaler_y.fit_transform(np.array(y_train).reshape(-1, 1))
# y_test_scaled = scaler_y.transform(np.array(y_test).reshape(-1, 1))

# new_predictions_scaled = np.maximum(new_predictions_scaled, 0)  

In [ ]:

predictions = scaler_y.inverse_transform(new_predictions_scaled)
predictions

Invert transformation


In [ ]:
# # # invert diff
# predictions_inverted_diff = np.cumsum(predictions)
# predictions_inverted_diff.shape

In [ ]:
# # # invert log
# predictions_inverted_log = np.exp(predictions_inverted_diff)
# predictions_inverted_log

In [ ]:
# # y_new = np.nan_to_num(y_new, nan=0.0, posinf=np.finfo(np.float32).max)
# predictions_inverted_log = np.nan_to_num(predictions_inverted_log, nan=0.0, posinf=np.finfo(np.float32).max)
# predictions_inverted_log

Evaluate the model


In [ ]:
loss = model.evaluate(X_test_lstm, y_test_scaled, verbose=1)
print(f"Test Loss: {loss:.4f}")
# print(f"Test Accuracy: {accuracy * 100:.2f}%")

Now its time for calculating the rmse performance matrix.


In [ ]:

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f"mean_squared_error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, color="blue", label="Predictions")
plt.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    "k--",
    lw=2,
    color="red",
    label="Perfect Prediction",
)
plt.xlabel("True Values (y_test)")
plt.ylabel("Predictions")
plt.title("True vs. Predicted Values")
plt.legend()
plt.show()

Invert tranformation


In [ ]:
# X_encoded["route_id_encoded"] = label_encoder.inverse_transform(X_encoded["route_id_encoded"])

# X_encoded

In [ ]:
results_df = pd.DataFrame(
    {
        "Actual": y_test,
        
        "Predicted": abs(predictions.flatten()),
        # "sid1": X_encoded["sid1"].values,
        # "sid2": X_encoded["sid2"].values,
        # "route_id": X_encoded["route_id_encoded"].values,
        # "Direction": X_new_encoded["direction_go"].values,
        # "hrs": X_new_encoded["hrs"].values,
        # "day_of_week": X_new_encoded["day_of_week"].values,
        # "time_periods": X_new_encoded["time_periods"].values,
    }
)

results_df[results_df['Predicted'] < 0]

In [ ]:
mae = mean_absolute_error(results_df["Actual"], results_df["Predicted"])
mae

### Try to find Feature importance


In [ ]:
import tensorflow as tf

# Assuming X is your DataFrame
X_values = X_encoded.values

# Reshape to add a time step dimension
X_values_reshaped = X_values.reshape((X_values.shape[0], 1, X_values.shape[1]))

# Convert to TensorFlow tensor
X_tensor = tf.convert_to_tensor(X_values_reshaped, dtype=tf.float32)

# Now you can use X_tensor with tape.watch()
with tf.GradientTape() as tape:
    tape.watch(X_tensor)
    y_pred = model(X_tensor)

# Compute gradients
gradients = tape.gradient(y_pred, X_tensor)

# Compute average impact
average_impact = tf.reduce_mean(tf.abs(gradients), axis=0).numpy()

In [ ]:
normalized_gradients = gradients / tf.reduce_max(tf.abs(gradients))
normalized_gradients.shape

In [ ]:
# Assuming normalized_gradients is a TensorFlow tensor
normalized_gradients_np = normalized_gradients.numpy().reshape(-1, 8)

plt.bar(features, np.mean(normalized_gradients_np, axis=0))
plt.xlabel("Feature Index")
plt.ylabel("Average Normalized Gradient")
plt.xticks(rotation=45, ha="right")
plt.show()